## Euro USD data set

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [3]:
data_usd_euro = pd.read_csv("EUR_USD.csv")
data_usd_euro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2862 entries, 0 to 2861
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tarih   2862 non-null   object
 1   Şimdi   2862 non-null   object
 2   Açılış  2862 non-null   object
 3   Yüksek  2862 non-null   object
 4   Düşük   2862 non-null   object
 5   Hac.    1033 non-null   object
 6   Fark %  2862 non-null   object
dtypes: object(7)
memory usage: 156.6+ KB


In [4]:
df_euro = data_usd_euro.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)

In [5]:
# We need to change from "," to "." because it will be a float. 
df_euro['Şimdi'] = df_euro['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '.')))
df_euro

,Tarih,Şimdi
0,19.12.2022,1.0601
1,18.12.2022,1.0584
2,16.12.2022,1.0582
3,15.12.2022,1.0626
4,14.12.2022,1.0682
...,...,...
2857,06.01.2012,1.2721
2858,05.01.2012,1.2780
2859,04.01.2012,1.2942
2860,03.01.2012,1.3051


## LSTM

In [6]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = df_euro['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[1.0601 1.0584 1.0582 1.0626 1.0682] 1.063
[1.0584 1.0582 1.0626 1.0682 1.063 ] 1.0535
[1.0582 1.0626 1.0682 1.063  1.0535] 1.053
[1.0626 1.0682 1.063  1.0535 1.053 ] 1.0556
[1.0682 1.063  1.0535 1.053  1.0556] 1.0505
[1.063  1.0535 1.053  1.0556 1.0505] 1.0469
[1.0535 1.053  1.0556 1.0505 1.0469] 1.0491
[1.053  1.0556 1.0505 1.0469 1.0491] 1.0538
[1.0556 1.0505 1.0469 1.0491 1.0538] 1.0522
[1.0505 1.0469 1.0491 1.0538 1.0522] 1.0405
[1.0469 1.0491 1.0538 1.0522 1.0405] 1.0327
[1.0491 1.0538 1.0522 1.0405 1.0327] 1.0337
[1.0538 1.0522 1.0405 1.0327 1.0337] 1.0395
[1.0522 1.0405 1.0327 1.0337 1.0395] 1.0408
[1.0405 1.0327 1.0337 1.0395 1.0408] 1.0395
[1.0327 1.0337 1.0395 1.0408 1.0395] 1.0302
[1.0337 1.0395 1.0408 1.0395 1.0302] 1.0241
[1.0395 1.0408 1.0395 1.0302 1.0241] 1.0324
[1.0408 1.0395 1.0302 1.0241 1.0324] 1.036
[1.0395 1.0302 1.0241 1.0324 1.036 ] 1.0392
[1.0302 1.0241 1.0324 1.036  1.0392] 1.0348
[1.0241 1.0324 1.036  1.0392 1.0348] 1.0325
[1.0324 1.036  1.0392 1.0348 1.0325

In [7]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df_euro['Şimdi'][0],df_euro['Şimdi'][1],df_euro['Şimdi'][2],df_euro['Şimdi'][3],df_euro['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Epoch 1/200
90/90 [==============================] - 1s 3ms/step - loss: 0.1455
Epoch 2/200
90/90 [==============================] - 0s 3ms/step - loss: 9.6963e-05
Epoch 3/200
90/90 [==============================] - 0s 3ms/step - loss: 8.6111e-05
Epoch 4/200
90/90 [==============================] - 0s 3ms/step - loss: 8.7489e-05
Epoch 5/200
90/90 [==============================] - 0s 4ms/step - loss: 9.0497e-05
Epoch 6/200
90/90 [==============================] - 0s 3ms/step - loss: 8.7379e-05
Epoch 7/200
90/90 [==============================] - 0s 3ms/step - loss: 8.9393e-05
Epoch 8/200
90/90 [==============================] - 0s 3ms/step - loss: 8.9252e-05
Epoch 9/200
90/90 [==============================] - 0s 3ms/step - loss: 8.7603e-05
Epoch 10/200
90/90 [==============================] - 0s 3ms/step - loss: 8.6466e-05
Epoch 11/200
90/90 [==============================] - 0s 3ms/step - loss: 8.9049e-05
Epoch 12/200
90/90 [==============================] - 0s 3ms/step - loss: 8.99

## Daily

In [8]:
print("Prediction:",round(yhat[0][0],3), "Euro/USD")# onluk olarak yukarı yuvarlandı.

Prediction: 1.066 Euro/USD


## Weekly

In [9]:
x_input = array([df_euro['Şimdi'][0], df_euro['Şimdi'][1], df_euro['Şimdi'][2], df_euro['Şimdi'][3], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df_euro['Şimdi'][0], df_euro['Şimdi'][1], df_euro['Şimdi'][2], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df_euro['Şimdi'][0], df_euro['Şimdi'][1],float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df_euro['Şimdi'][0], float(yhat_4),float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)

x_input = array([float(yhat_5), float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_6 = model.predict(x_input, verbose=0)

yhat_6 = float(yhat_6)


In [10]:
print(f"Prediction -> Pazartesi:{yhat} Salı:{yhat_2} Carsamba:{yhat_3} Persembe:{yhat_4} Cuma:{yhat_5}  $ Euro/USD ")


Prediction -> Pazartesi:[[1.065979]] Salı:[[1.0652862]] Carsamba:[[1.0658573]] Persembe:[[1.0672075]] Cuma:[[1.0685053]]  $ Euro/USD 


In [11]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Week -> Prediction: {round(yhat_6,3)} $ Euro/USD ")


Next Week -> Prediction: 1.07 $ Euro/USD 


## Monthly

In [12]:
import array
exchange = array.array('f', [])

for i in range(len(df_euro)):
    if i % 5 == 0:  # it equals to weekly
        exchange.append(df_euro['Şimdi'][i])


In [13]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.0601 1.063  1.0469 1.0327 1.0302] 1.0348000526428223
[1.063  1.0469 1.0327 1.0302 1.0348] 1.007200002670288
[1.0469 1.0327 1.0302 1.0348 1.0072] 0.9873999953269958
[1.0327 1.0302 1.0348 1.0072 0.9874] 0.996399998664856
[1.0302 1.0348 1.0072 0.9874 0.9964] 0.9851999878883362
[1.0348 1.0072 0.9874 0.9964 0.9852] 0.970300018787384
[1.0072 0.9874 0.9964 0.9852 0.9703] 0.9983000159263611
[0.9874 0.9964 0.9852 0.9703 0.9983] 0.9592000246047974
[0.9964 0.9852 0.9703 0.9983 0.9592] 0.996999979019165
[0.9852 0.9703 0.9983 0.9592 0.997 ] 0.996999979019165
[0.9703 0.9983 0.9592 0.997  0.997 ] 0.9901999831199646
[0.9983 0.9592 0.997  0.997  0.9902] 1.0011999607086182
[0.9592 0.997  0.997  0.9902 1.0012] 0.9966999888420105
[0.997  0.997  0.9902 1.0012 0.9967] 1.0169999599456787
[0.997  0.9902 1.0012 0.9967 1.017 ] 1.0211000442504883
[0.9902 1.0012 0.9967 1.017  1.0211] 1.0163999795913696
[1.0012 0.9967 1.017  1.0211 1.0164] 1.0113999843597412
[0.9967 1.017  1.0211 1.0164 1.0114] 1.02240002155303

In [14]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/200
18/18 [==============================] - 1s 5ms/step - loss: 0.5038
Epoch 2/200
18/18 [==============================] - 0s 5ms/step - loss: 0.0318
Epoch 3/200
18/18 [==============================] - 0s 5ms/step - loss: 0.0046
Epoch 4/200
18/18 [==============================] - 0s 5ms/step - loss: 0.0011
Epoch 5/200
18/18 [==============================] - 0s 5ms/step - loss: 5.0315e-04
Epoch 6/200
18/18 [==============================] - 0s 5ms/step - loss: 4.1676e-04
Epoch 7/200
18/18 [==============================] - 0s 5ms/step - loss: 3.9051e-04
Epoch 8/200
18/18 [==============================] - 0s 5ms/step - loss: 3.9275e-04
Epoch 9/200
18/18 [==============================] - 0s 5ms/step - loss: 3.9060e-04
Epoch 10/200
18/18 [==============================] - 0s 5ms/step - loss: 3.9198e-04
Epoch 11/200
18/18 [==============================] - 0s 5ms/step - loss: 3.9041e-04
Epoch 12/200
18/18 [==============================] - 0s 5ms/step - loss: 3.9322e-04
Epoch

In [15]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Month -> Prediction: {round(float(yhat[0][0]),4)} Euro/USD ")


Next Month -> Prediction: 1.0588 Euro/USD 


## Yearly

In [16]:
import array
exchange_2 = array.array('f', [])

for i in range(len(df_euro)):
    if i % 30 == 0:
        exchange_2.append(df_euro['Şimdi'][i])


In [17]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange_2
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.0601 1.0072 0.9592 1.017  1.0266] 1.0734000205993652
[1.0072 0.9592 1.017  1.0266 1.0734] 1.0825999975204468
[0.9592 1.017  1.0266 1.0734 1.0826] 1.1125999689102173
[1.017  1.0266 1.0734 1.0826 1.1126] 1.1325000524520874
[1.0266 1.0734 1.0826 1.1126 1.1325] 1.1262999773025513
[1.0734 1.0826 1.1126 1.1325 1.1263] 1.159500002861023
[1.0826 1.1126 1.1325 1.1263 1.1595] 1.1801999807357788
[1.1126 1.1325 1.1263 1.1595 1.1802] 1.1859999895095825
[1.1325 1.1263 1.1595 1.1802 1.186 ] 1.1937999725341797
[1.1263 1.1595 1.1802 1.186  1.1938] 1.2145999670028687
[1.1595 1.1802 1.186  1.1938 1.2146] 1.1713999509811401
[1.1802 1.186  1.1938 1.2146 1.1714] 1.2103999853134155
[1.186  1.1938 1.2146 1.1714 1.2104] 1.2294000387191772
[1.1938 1.2146 1.1714 1.2104 1.2294] 1.1887999773025513
[1.2146 1.1714 1.2104 1.2294 1.1888] 1.174399971961975
[1.1714 1.2104 1.2294 1.1888 1.1744] 1.190999984741211
[1.2104 1.2294 1.1888 1.1744 1.191 ] 1.1526000499725342
[1.2294 1.1888 1.1744 1.191  1.1526] 1.133999943733

In [18]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)

# demonstrate prediction
x_input = array([exchange_2[0], exchange_2[1],
                exchange_2[2], exchange_2[3], exchange_2[4]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/200
3/3 [==============================] - 1s 6ms/step - loss: 1.3054
Epoch 2/200
3/3 [==============================] - 0s 6ms/step - loss: 0.9194
Epoch 3/200
3/3 [==============================] - 0s 5ms/step - loss: 0.5836
Epoch 4/200
3/3 [==============================] - 0s 5ms/step - loss: 0.2747
Epoch 5/200
3/3 [==============================] - 0s 6ms/step - loss: 0.0364
Epoch 6/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0955
Epoch 7/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1042
Epoch 8/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0126
Epoch 9/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0192
Epoch 10/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0406
Epoch 11/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0332
Epoch 12/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0124
Epoch 13/200
3/3 [==============================] - 0s 5ms/st

In [19]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Year -> Prediction: {round(float(yhat[0][0]),4)} Euro/USD ")


Next Year -> Prediction: 1.0469 Euro/USD 
